In [7]:
import ipywidgets as widgets
from bs4 import BeautifulSoup
from selenium import webdriver

text_layout = widgets.Layout(width='1000',
                align_items='center')

sqinput = widgets.Text(
            description='Query:',
            placeholder = "Try to input small home 3d printer hotbed extruder in box",
            width = 10,
            layout = text_layout)
vecsize = widgets.IntSlider(
            value=125,
            min=1,
            max=200,
            step=1,
            description='Vector Size:',
            orientation='horizontal',
            readout=True,
            readout_format='d',
            width = 500
            )
epochsize = widgets.IntSlider(
            value=200,
            min=100,
            max=300,
            step=1,
            description='Epoch Size:',
            orientation='horizontal',
            readout=True,
            readout_format='d',
            width = 500
            )
searchbutton = widgets.Button(description="Search")
button1 = widgets.Button(description="1")
button2 = widgets.Button(description="2")
button3 = widgets.Button(description="3")
button4 = widgets.Button(description="4")
button5 = widgets.Button(description="5")

In [8]:
import csv
import re
import nltk
import gensim
import random
from IPython.display import Image,HTML
from IPython import display

def main(but):
    but = True
    search_query = sqinput.value
    def get_text(dataset):
        words = set(nltk.corpus.words.words())
        rawtext = []
        rawtextph = []
        with open(r'C:\Users\ASUS\.ipynb_checkpoints\{}.csv'.format(dataset), 'r', encoding='utf-8') as read_obj:
            reader = csv.reader(read_obj)
            next(reader, None)
            for row in reader:
                if row[7] != "[]":
                    title = row[0]
                    title = " ".join(w for w in nltk.wordpunct_tokenize(title) if w.lower() in words or not w.isalpha())
                    title = re.sub('[^A-Za-z0-9]+', ' ', title)
                    rawtextph.append(title)
                    func = row[2]
                    func = " ".join(w for w in nltk.wordpunct_tokenize(func) if w.lower() in words or not w.isalpha())
                    func = re.sub('[^A-Za-z0-9]+',' ',func)
                    revs = row[7]
                    revs = " ".join(w for w in nltk.wordpunct_tokenize(revs) if w.lower() in words or not w.isalpha())
                    revs = re.sub('[^A-Za-z0-9]+', ' ', revs)
                    combined = func + revs
                    rawtextph.append(combined)
                    url = row[5]
                    rawtextph.append(url)
                    rawtitle = row[0]
                    rawtextph.append(rawtitle)
                    rawfunc = row[2]
                    rawtextph.append(rawfunc)
                    rawtext.append(rawtextph)
                    rawtextph = []
        return rawtext

    def read_corpus(fname, tokens_only=False):
        corpora = []
        for row in fname:
            corpora.append(row[1])
        for i, line in enumerate(corpora):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

    rawtext = get_text('datasetnorating')

    corpus = list(read_corpus(rawtext))
    sampled = []
    for x in corpus:
        for y in range(0,10):
            randomsample = random.sample(x[0],7)
            sampled.append(gensim.models.doc2vec.TaggedDocument(randomsample, x[1]))

    trainlen = round(0.8*len(sampled))
    testlen = round(0.2*len(sampled))
    trainset = random.sample(sampled,trainlen)
    testset = random.sample(sampled,testlen)

    model = gensim.models.doc2vec.Doc2Vec(vector_size=vecsize.value, min_count=2, epochs=epochsize.value)
    model.build_vocab(trainset)
    model.train(trainset, total_examples=model.corpus_count, epochs=model.epochs)

    # SEARCH ENGINE PART
    mostsimurls = []
    mostsimids = []
    mostsimimgs = []
    mostsimtitles = []
    mostsimfuncs = []
    sqtokens = gensim.utils.simple_preprocess(sqinput.value)
    sq_vector = model.infer_vector(sqtokens)
    sims = model.dv.most_similar([sq_vector], topn=len(model.dv))
    for x in range(0,5):
        mostsimids.append(sims[x][0])
    for y in mostsimids:
        mostsimurls.append(rawtext[y][2])
        mostsimtitles.append(rawtext[y][3])
        mostsimfuncs.append(rawtext[y][4])
    driver = webdriver.Chrome()
    for z in mostsimurls:
        driver.get(z)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        imagestr = str(soup.find('img', id='landingImage'))
        imagestr = imagestr.split('src="')[1]
        imagestr = imagestr.split('"')[0]
        mostsimimgs.append(imagestr)
    driver.close()
    toptext = widgets.HTML(value="<h5>Top 5 Recommendations based on '{}' : </h5>".format(search_query))
    display.display(toptext)
    tab_contents = ['1', '2', '3', '4', '5']
    children = [widgets.VBox([widgets.HTML(value="Recommendation No. {} \n".format(int(name))),
                              widgets.HTML(value="\nProduct Title : {}\n".format(mostsimtitles[int(name)-1])),
                              widgets.HTML(value="\nProduct Function : {}\n".format(mostsimfuncs[int(name)-1])),
                              widgets.HTML(value="\n<img src={}>".format(mostsimimgs[int(name)-1]))])for name in tab_contents
                ]
    tab = widgets.Tab()
    tab.children = children
    for i in range(5):
        tab.set_title(i, str(i+1))
    display.display(tab)

In [9]:
from IPython import display
from ipywidgets import VBox,HBox
import ipywidgets as widgets
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center')
inputbox = VBox([widgets.HTML(value="<h1> <center> Online Product Configuration Platform </center> </h1>"),sqinput,vecsize,epochsize,searchbutton],layout=box_layout)
box_layout = widgets.Layout(display='flex',
                align_items='center')
searchbutton.on_click(main)
display.display(inputbox,layout=box_layout)